In [1]:
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import ast

In [4]:
output_Steam_Games = os.path.join('Datasets-Limpios', 'output_games.parquet')
games = pd.read_parquet(output_Steam_Games)
modelo_df = games.loc[:, ["genres", "item_id",'tags']]

In [5]:
genres_variable = modelo_df['genres'].copy()
genres_variable = genres_variable.apply(lambda x: str(x).replace('[', '').replace(']', ''))
genres_variable = genres_variable.apply(lambda x: ', '.join(filter(None, map(str.strip, x.split("'")))) if isinstance(x, str) else x)
modelo_df['genres'] = genres_variable
print(modelo_df['genres'])

0              Action, Casual, Indie, Simulation, Strategy
1                       Free to Play, Indie, RPG, Strategy
2          Casual, Free to Play, Indie, Simulation, Sports
3                                Action, Adventure, Casual
4                            Action, Indie, Casual, Sports
                               ...                        
31989                  Casual, Indie, Simulation, Strategy
31990                              Casual, Indie, Strategy
31991                            Indie, Racing, Simulation
31992                                        Casual, Indie
31993    Early Access, Adventure, Indie, Action, Simula...
Name: genres, Length: 31994, dtype: object


In [6]:
modelo_df['genres'] = modelo_df['genres'].replace('Design &amp; Illustration', 'Design & Illustration')
modelo_df['genres'] = modelo_df['genres'].replace('Animation &amp; Modeling', 'Animation & Modeling')

In [7]:
print(modelo_df['genres'].unique())

['Action, Casual, Indie, Simulation, Strategy'
 'Free to Play, Indie, RPG, Strategy'
 'Casual, Free to Play, Indie, Simulation, Sports' ...
 'Action, Massively Multiplayer, Strategy'
 'Video Production, Utilities, Web Publishing'
 'Early Access, Adventure, Indie, Action, Simulation']


In [8]:
# cv es una instancia de la clase CountVectorizer de scikit-learn, utilizada para convertir documentos de texto en una matriz de recuentos de términos, donde cada fila representa un documento y cada columna una palabra única.
cv = CountVectorizer()

In [9]:
# Ajustar CountVectorizer a la columna 'genres' y transformar los datos
# .toarray(): Convertir la representación dispersa de la matriz de recuentos a una matriz densa.
# .shape: Obtener la forma de la matriz resultante (número de filas y columnas).
cv.fit_transform(modelo_df['genres']).toarray().shape

(31994, 33)

In [10]:
# Se generan los vectores para comparar
vectores = cv.fit_transform(modelo_df['genres']).toarray()

In [11]:
vectores

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0]])

In [12]:
similitud = cosine_similarity(vectores)

In [13]:
similitud[0]

array([1.        , 0.36514837, 0.50709255, ..., 0.51639778, 0.63245553,
       0.54772256])

In [14]:
# Ordenamos la similitud entre más similar a menos similar tomando 5 valores
sorted(list(enumerate(similitud[0])), reverse=True, key=lambda x:x[1])[1:6]

[(3336, 0.9999999999999999),
 (3553, 0.9999999999999999),
 (5340, 0.9999999999999999),
 (5690, 0.9999999999999999),
 (6903, 0.9999999999999999)]

In [15]:
def recomendacion(id_juego):
    # Obtenemos el índice del juego con el ID dado
    indice_juego = modelo_df[modelo_df["item_id"] == id_juego].index[0]

    # Obtenemos las distancias de similitud para el juego dado
    distances = similitud[indice_juego]

    # Ordenamos las distancias en orden descendente y obtener las 5 recomendaciones principales (excluyendo el juego dado)
    lista_juegos = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    # Obtenemos los nombres de los juegos recomendados
    recommended_titles = [modelo_df.iloc[i[0]]['tags'] for i in lista_juegos]

    return recommended_titles

In [18]:
# Se aplica la función al dataframe para obtener una nueva columna con las recomendaciones ya que es más facil
# de cargar y leer para la funcion
modelo_df['recomendaciones_top_5'] = modelo_df['item_id'].apply(recomendacion)

In [19]:
modelo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31994 entries, 0 to 31993
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   genres                 31994 non-null  object
 1   item_id                31994 non-null  int32 
 2   tags                   31994 non-null  object
 3   recomendaciones_top_5  31994 non-null  object
dtypes: int32(1), object(3)
memory usage: 875.0+ KB


In [20]:
modelo_df.drop(columns=['tags','genres'], inplace=True)

In [21]:
modelo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31994 entries, 0 to 31993
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   item_id                31994 non-null  int32 
 1   recomendaciones_top_5  31994 non-null  object
dtypes: int32(1), object(1)
memory usage: 375.1+ KB


In [22]:
modelo_df.to_csv('consulta6.csv', index=False)